In [72]:
import json
from os.path import join
from SPARQLWrapper import SPARQLWrapper, JSON
from string import Template
from tqdm import tqdm

In [73]:
outputFolder = "../data/iiif/"

In [74]:
endpoint = "http://localhost:7776/blazegraph/sparql"

In [75]:
def sparqlResultToDict(results):
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for key in list(result.keys()):
            row[key] = result[key]["value"]
        rows.append(row)
    return rows


In [76]:
sparql = SPARQLWrapper(endpoint)
sparql.setReturnFormat(JSON)

In [77]:
objectsQuery = """
PREFIX search: <https://platform.swissartresearch.net/search/>
SELECT ?subject ?label WHERE {
    ?subject a search:Object ;
        rdfs:label ?label .
}
"""

In [78]:
sparql.setQuery(objectsQuery)
objects = sparqlResultToDict(sparql.query().convert())

In [79]:
manifests = []

imageQueryTemplate = Template("""
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX la: <https://linked.art/ns/terms/>
SELECT ?image ?width ?height WHERE {
    <$uri> crm:P128_carries/la:digitally_shown_by ?imageObject .
    ?imageObject la:digitally_available_via/la:access_point ?image ;
        crm:P43_has_dimension ?dimWidth ;
        crm:P43_has_dimension ?dimHeight .
    ?dimWidth crm:P2_has_type aat:300055647 ;
        crm:P90_has_value ?width .
    ?dimHeight crm:P2_has_type aat:300055644 ;
        crm:P90_has_value ?height .
}
""")

for obj in tqdm(objects):

    manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": "%s/manifest.json" % obj['subject'],
        "items": [],
        "type": "Manifest",
        "label": {
            "en": [obj['label']]
        }
    }
    
    imageQuery = imageQueryTemplate.substitute(uri=obj['subject'])
    sparql.setQuery(imageQuery)
    images = sparqlResultToDict(sparql.query().convert())
    
    for i, image in enumerate(images):
        canvas = {
            "id": "%s/image/%d/canvas" % (obj['subject'], i),
            "type": "Canvas",
            "width": int(image['width']),
            "height": int(image['height']),
            "items": [{
                    "id": "%s/image/%d/canvas/page" % (obj['subject'], i),
                    "type": "AnnotationPage",
                    "items": [{
                        "id": "%s/image/%d/canvas/page/annotation" % (obj['subject'], i),
                        "type": "Annotation",
                        "motivation": "painting",
                        "body": {
                            "id": "%s/full/max/0/default.jpg" % image['image'],
                            "type": "Image",
                            "format": "image/jpeg",
                            "width": int(image['width']),
                            "height": int(image['height']),
                            "service": [{
                                "id": image['image'],
                                "profile": "level1",
                                "type": "ImageService3"
                            }]
                        },
                        "target": "%s/image/%d/canvas" % (obj['subject'], i)
                    }]
                }]
        }
        manifest['items'].append(canvas)
    manifests.append(manifest)

100%|██████████| 40/40 [00:00<00:00, 63.48it/s]


In [80]:
for manifest in manifests:
    filename = join(outputFolder, manifest['id'][len("https://resource.swissartresearch.net/artwork/"):-len("/manifest.json")] + ".json")
    with open(filename, 'w') as f:
        json.dump(manifest, f, ensure_ascii=False, indent=4)